In [1]:
import Pkg;
Pkg.activate(joinpath(@__DIR__, ".."));
Pkg.instantiate();
using DelimitedFiles
using CSV
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Random;

  Activating environment at `~/planar_quadruped_landing/Project.toml`


In [2]:
include("quadratic_cost.jl")
include("planar_quadruped.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl");

In [3]:
h = 2
v_init_y = sqrt(2 * 9.81 * h)

6.26418390534633

In [4]:
# Dynamics model
model = PlanarQuadruped()
g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

# Some parameters
dt = 0.009
N = 41
times = range(0, dt * (N - 1), length=N)
n = 15
m = 5

# initial state
xinit = zeros(n)
xinit[1] = -lb / 2            # xb
xinit[2] = sqrt(l1^2 + l2^2)  # yb
xinit[6] = -lb                # x2
xinit[9] = -v_init_y          # yb_dot
xinit[10] = -pi/2             # theta_dot

# Desired final state
xterm = zeros(n)
xterm[1] = -lb / 2           # xb
xterm[2] = sqrt(l1^2 + l2^2) # yb
xterm[6] = -lb;              # x2

In [5]:
# Reference Trajectory
Xref, Uref = reference_trajectory(model, N, xterm, dt);

In [6]:
# Objective
Q = Diagonal([10.0; 10.0; 1.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 1.0; 10.0; 10.0; 10.0; 10.0; 0.0])
R = Diagonal(fill(1e-3, 5))
R[end, end] = 0.0
Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q, R, Xref[k], Uref[k])
end
push!(obj, LQRCost(Qf, R * 0, Xref[N], Uref[1]));

In [7]:
# Define the NLP
nlp = HybridNLP(model, obj, N, xinit, xterm);

In [8]:
# initialize guess
Xguess = [zeros(n) for x in Xref]

for k = 1:N
    Xguess[k][1:14] = xterm[1:14]
    Xguess[k][end] = dt * (k - 1)
end

In [12]:
Z0 = packZ(nlp, Xguess, Uref)

Z_sol, solver = solve(Z0, nlp, c_tol=1e-3, tol=1e-3)

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...


This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   580280
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      815
                     variables with only lower bounds:       80
                variables with lower and upper bounds:       81
                     variables with only upper bounds:        0
Total number of equality constraints.................:      712
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

([-0.25000000000000044, 0.3535533905932738, 5.0127396250101924e-18, -4.997135112800288e-9, 3.548562252082801e-19, -0.49999999999999983, -1.0874446350183664e-18, -1.757182583283248e-17, -6.26418390534633, -1.5707963267948966  …  -0.50000000000003, 1.3352632358296945e-17, 2.6160144182112552e-17, -7.223170926082834e-21, 2.3837146695059082e-18, 0.0, 0.0, 0.0, 0.0, 0.6645707693199359], Ipopt.Optimizer)

In [ ]:
Z_sol[1:15] - xinit

In [ ]:
Z_sol[end-14:end] - xterm

In [ ]:
nlp.uinds[end]

In [ ]:
@show F1y_final = Z_sol[797]
@show F2y_final = Z_sol[799]
@show F1y_final + F2y_final

In [ ]:
dt = zeros(N - 1)
for k = 1:N-1
    @show dt[k] = Z_sol[20+20*(k-1)];
end

In [ ]:
F1y = zeros(N-1)
F2y = zeros(N-1)
for k = 1:N-1
    @show F1y[k] = Z_sol[17+20*(k-1)];
    @show F2y[k] = Z_sol[19+20*(k-1)];
end

In [ ]:
ui = nlp.uinds
Z_sol[ui[end]][2] + Z_sol[ui[end]][4] + nlp.model.mb * nlp.model.g

In [ ]:
# display(Z_sol)
writedlm("data_8.csv", Z_sol, ',')